In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install 'crewai[tools]'
!pip install --upgrade crewai crewai-tools
!pip freeze | grep crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 676.2 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.9/514.9 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.9 MB/s

In [28]:
import google.generativeai as genai
llm = genai.GenerativeModel("gemini-2.0-flash-exp")

In [4]:
from crewai import Agent, Task, Crew

In [5]:
from google.colab import userdata

# Retrieve the Google API key from Colab secrets
google_api_key = userdata.get('GOOGLE_API_KEY_2')

# Confirm retrieval (Optional, for debugging purposes)
if google_api_key:
    print("Google API key retrieved successfully.")
else:
    print("Failed to retrieve the API key. Make sure it's stored as 'GOOGLE_API_KEY'.")


Google API key retrieved successfully.


In [ ]:
# # Initialize Google PaLM 2 LLM
# llm = GooglePalm(api_key=google_api_key)

In [6]:
import os

# Set the Google API key as an environment variable
os.environ["GOOGLE_API_KEY"] = google_api_key


### Agent: Senior Data Scientist (Lead Agent)

In [7]:
senior_data_scientist = Agent(
    role="Senior Data Scientist (Lead Agent)",
    goal="Oversee and guide the entire stock market forecasting pipeline, "
         "ensuring high-quality results and actionable insights for clients.",
    backstory="You are the leader of the forecasting team, responsible for "
              "developing and optimizing models, mentoring team members, "
              "and ensuring the accuracy of forecasts. "
              "You coordinate with other agents to ensure data integrity, "
              "statistical soundness, and client-specific requirements are met. "
              "Your expertise includes advanced time series forecasting "
              "and integrating domain knowledge into technical solutions.",
    allow_delegation=True,
    verbose=True
)

### Agent: Stock Market Analyst (Research Specialist)

In [8]:
stock_market_analyst = Agent(
    role="Stock Market Analyst (Research Specialist)",
    goal="Conduct in-depth research on stock market trends, patterns, "
         "and anomalies to provide actionable insights.",
    backstory="You specialize in understanding market dynamics, analyzing "
              "financial statements, and identifying patterns or irregularities "
              "that could impact forecasting. "
              "You work closely with the Data Scientist to provide contextual insights "
              "that enhance the predictive models. "
              "You are skilled in interpreting complex financial data "
              "and communicating findings effectively.",
    allow_delegation=False,
    verbose=True
)

### Agent: Financial Operations Coordinator (Client Liaison)

In [9]:
financial_operations_coordinator = Agent(
    role="Financial Operations Coordinator (Client Liaison)",
    goal="Act as the primary point of contact for clients, ensuring "
         "clear communication and alignment with their objectives.",
    backstory="You manage client relationships, translate their needs into "
              "technical requirements, and ensure timely delivery of insights. "
              "Your role involves setting priorities for the team based on "
              "client input and ensuring the deliverables meet their standards. "
              "You also provide feedback from clients to the team to refine the process.",
    allow_delegation=False,
    verbose=True
)

### Agent: Data Operations Specialist (Data Integrator)

In [10]:
data_operations_specialist = Agent(
    role="Data Operations Specialist (Data Integrator)",
    goal="Integrate and preprocess stock market data from multiple sources, "
         "ensuring it is clean, complete, and ready for analysis.",
    backstory="You are responsible for managing data pipelines, handling "
              "API integrations, and resolving data quality issues. "
              "Your expertise lies in extracting, transforming, and loading (ETL) "
              "data to create a seamless flow of information. "
              "You work closely with the Data Ingestion Agent and Senior Data Scientist "
              "to maintain data integrity and reliability.",
    allow_delegation=False,
    verbose=True
)


### Agent: Model Trainer Agent

In [11]:
model_trainer_agent = Agent(
    role="Model Trainer Agent",
    goal="Train and evaluate stock market forecasting models using historical data.",
    backstory="You are responsible for selecting, training,"
              "and evaluating machine learning models for "
              "stock market forecasting. Using the pre-processed "
              "data provided by the Data Operations Specialist, you'll"
              "apply various models such as ARIMA, SARIMA, LSTM, and others,"
              "and fine-tune them for optimal performance. You'll also evaluate"
              "the models using appropriate metrics like RMSE, MAE, and MAPE. Once you’ve"
              "trained the model, you'll collaborate with the Financial Operations Coordinator"
              "to ensure it’s ready for deployment in the forecasting pipeline.",
    allow_delegation=False,  # Model training requires direct involvement and cannot be delegated
    verbose=True
)


### Agent: Model Evaluator

In [12]:
model_evaluator_agent = Agent(
    role="Model Evaluator Agent",
    goal="Evaluate and assess the performance of forecasting models to ensure their accuracy and reliability.",
    backstory="You are tasked with evaluating the performance of forecasting models"
              "after they've been trained. You will use various performance metrics such as RMSE,"
              "MAE, MAPE, and others to assess the accuracy of the models. Your job is to perform a"
              "comprehensive evaluation and provide insights on the model's strengths and weaknesses."
              "You'll collaborate with the Model Trainer Agent to ensure the models are refined and ready"
              "for deployment. You will also provide detailed feedback and help make final decisions about"
              "model deployment in coordination with the Senior Data Scientist and other stakeholders.",
    allow_delegation=False,  # Evaluation requires direct involvement
    verbose=True
)


### Agent: Visualisation Agent

In [13]:
visualization_agent = Agent(
    role="Visualization Agent",
    goal="Create and present clear, insightful visualizations of the model's forecasting results, trends, and insights.",
    backstory="You are responsible for generating visualizations that highlight"
              "the key trends and insights derived from the forecasting models. This"
              "includes creating time series plots, error analysis charts, comparison plots,"
              "and any other visuals that help stakeholders understand the model's performance"
              "and forecasted results. You will work closely with the Model Evaluator and Senior"
              "Data Scientist to create visual representations that accurately convey important findings"
              "in a clear and understandable manner. Your visualizations should be tailored to various stakeholders including business leaders, analysts, and clients.",
    allow_delegation=False,  # Visualization is a hands-on task requiring direct involvement
    verbose=True
)


### Task for Data Operations Specialist (Data Integrator)

In [14]:
data_integration_task = Task(
    description="Integrate and preprocess stock market data from multiple sources, ensuring it is clean, complete, and ready for analysis.",
    expected_output="A preprocessed dataset containing clean, complete, and consistent stock market data, ready for further analysis.",
    agent=data_operations_specialist
)


### Task for Stock Market Analyst (Research Specialist)

In [15]:
research_task = Task(
    description="Analyze stock market trends, patterns, and anomalies to provide actionable insights for modeling.",
    expected_output="A report containing trends, anomalies, and insights relevant to stock market forecasting.",
    agent=stock_market_analyst
)


### Task for Model Trainer Agent

In [16]:
model_training_task = Task(
    description="Train and fine-tune forecasting models using the preprocessed stock market data and insights from the analyst.",
    expected_output="Trained forecasting models optimized for performance, along with training logs and initial evaluation metrics.",
    agent=model_trainer_agent
)


### Task for Model Evaluator Agent

In [17]:
model_evaluation_task = Task(
    description="Evaluate the performance of trained models using appropriate metrics like RMSE, MAE, and MAPE.",
    expected_output="Detailed evaluation reports for each trained model, highlighting strengths, weaknesses, and recommendations for deployment.",
    agent=model_evaluator_agent
)


### Task for Visualization Agent

In [18]:
visualization_task = Task(
    description="Create visualizations to present model results, trends, and insights clearly to stakeholders.",
    expected_output="Visualizations including time series plots, trend analysis charts, and performance comparison plots for stakeholders.",
    agent=visualization_agent
)


### Task for Senior Data Scientist (Lead Agent)

In [19]:
senior_data_scientist_task = Task(
    description="Oversee and guide the forecasting pipeline, ensuring all outputs meet quality standards and are aligned with client requirements.",
    expected_output="Final report containing actionable insights, validated models, and a comprehensive summary of findings for client delivery.",
    agent=senior_data_scientist
)


### Task for Financial Operations Coordinator (Client Liaison)

In [20]:
client_communication_task = Task(
    description="Communicate results, visualizations, and insights to the client, ensuring alignment with their objectives.",
    expected_output="Client-facing report with insights, visualizations, and recommendations tailored to their specific requirements.",
    agent=financial_operations_coordinator
)


### Task Pipeline

In [21]:
# tasks = [
#     data_integration_task,
#     research_task,
#     model_training_task,
#     model_evaluation_task,
#     visualization_task,
#     senior_data_scientist_task,
#     client_communication_task
# ]


### Creating the Crew

In [23]:
crew = Crew(
    agents=[
        data_operations_specialist,
        stock_market_analyst,
        model_trainer_agent,
        model_evaluator_agent,
        visualization_agent,
        senior_data_scientist,
        financial_operations_coordinator
    ],
    tasks=[
        data_integration_task,
        research_task,
        model_training_task,
        model_evaluation_task,
        visualization_task,
        senior_data_scientist_task,
        client_communication_task
    ],
    verbose=True
)


In [24]:
inputs = {
    "asset": "US WTI Crude Oil (USOIL)",  # Asset to forecast
    "intervals": ["15 mins", "30 mins", "1 hour", "3 hours", "1 day", "3 days", "weekly"],  # Forecasting intervals
    "historical_data_source": "TradingView or alternative API",  # Historical data source for USOIL prices
    "data_requirements": {
        "years_of_historical_data": 5,  # Train model on several years of historical data
        "short_window_for_predictions": 50  # Use a 50-day sliding window for predictions
    },
    "model": {
        "type": "LSTM",  # Model type for forecasting
        "accuracy_goal": 90,  # Target accuracy
        "evaluation_metrics": ["MAE", "RMSE", "R2"]  # Metrics for model evaluation
    },
    "daily_tasks": {
        "plot_comparison": "Plot actual vs predicted prices daily",
        "save_predictions": "Store daily predicted prices in a file for iterative plots"
    },
    "timeline": {
        "delivery_days": 7,  # Timeframe for completing the project
        "priority": "High-quality over rushed output"
    },
    "visualization": "Interactive charts to compare actual and predicted prices across intervals",
    "quote_discussion": "Discuss cost and effort for delivering this solution",
    "deployment_notes": {
        "training_frequency": "Train once on historical data",
        "inference_frequency": "Perform daily predictions with sliding window"
    }
}


In [29]:
# Kick off the crew
result = crew.kickoff(inputs=inputs)

ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


# Agent: Data Operations Specialist (Data Integrator)
## Task: Integrate and preprocess stock market data from multiple sources, ensuring it is clean, complete, and ready for analysis.


LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
from IPython.display import Markdown
Markdown(result)

In [ ]:
# Initialize Google PaLM 2 LLM
llm = GooglePalm(api_key=google_api_key)